Import the libraries needed

In [1]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import psutil


import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


C:\Users\m033\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


### Database connections and SQL to retrieve the various data needed.

In [2]:
SQL_server = 'xxx\yyy'
Reporting_database = 'zzz'
Reference_Data_database = 'aaa'

pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-08-14 17:54:43.094719
svmem(total=16756752384, available=4414054400, percent=73.7, used=12342697984, free=4414054400)


# 1. Get supplementary policy features - Premium and Indexation.

  -  Identified in Lit Review.# 1. Read back the population with switches data.

In [3]:
conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+SQL_server+';'
         'DATABASE='+Reporting_database+';'
         'Trusted_Connection=yes;')

premium_query = f"""SELECT pol.Policy_Number                AS Contract_Number
	                      ,pol.Premiums_Per_Year            AS PayFreq
	                      ,pol.Anniversary_Date             AS AnniversaryDt
	                      ,pol.Indexation_Percentage        AS IndexationPercent 
	                      ,pol.Indexation_Start_Date        AS IndexationStartDt 
	                      ,pol.Premium_Holiday_Start        AS PremHolStartDt
	                      ,pol.Premium_Holiday_End          AS PremHolEndDt
	                      ,pol.Regular_Premium              AS Premium
                          ,pol.Premium_Calc_Method          AS PremiumCalcMethod
                 FROM Policy pol
                 INNER JOIN Product prd ON prd.Product_Id = pol.Product_Id
                 WHERE prd.Product_Code IN ('FKP','GNPP_','GNRP_1')
                 ORDER BY pol.Policy_Number;"""

df_premium = pd.read_sql(premium_query, conn)

# Types and row count
# print stats.
print(len(df_premium.index))
now = datetime.now()
print(now)
print('==========================')

print(df_premium.dtypes)
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_premium.head(5)

137245
2024-08-14 17:55:03.749175
Contract_Number       object
PayFreq                int64
AnniversaryDt         object
IndexationPercent    float64
IndexationStartDt     object
PremHolStartDt        object
PremHolEndDt          object
Premium              float64
PremiumCalcMethod     object
dtype: object
2024-08-14 17:55:03.750254
svmem(total=16756752384, available=4382179328, percent=73.8, used=12374573056, free=4382179328)


## Drop obsolete or repeated features.

## Check for nulls

In [4]:
df_premium.dtypes

Contract_Number       object
PayFreq                int64
AnniversaryDt         object
IndexationPercent    float64
IndexationStartDt     object
PremHolStartDt        object
PremHolEndDt          object
Premium              float64
PremiumCalcMethod     object
dtype: object

In [5]:
df_premium.isna().any()

Contract_Number      False
PayFreq              False
AnniversaryDt         True
IndexationPercent     True
IndexationStartDt     True
PremHolStartDt        True
PremHolEndDt          True
Premium              False
PremiumCalcMethod    False
dtype: bool

## Replace any nulls before checking the distributions of values for each column.

In [6]:
df_premium['IndexationPercent'] = df_premium['IndexationPercent'].fillna(0).apply(np.int64)
df_premium['IndexationPercent'] = df_premium['IndexationPercent'].fillna(0).apply(np.int64)

## Check the distributions of values for each column.

In [7]:
print('IndexationPercent : ',df_premium['IndexationPercent'].value_counts())

IndexationPercent :  0    137245
Name: IndexationPercent, dtype: int64


## Count does not show it but IndexationPercent has a distribution so leave it in.

# write to Excel

In [8]:
# Write out to a comma separated values file.

# giving directory name
filename = input_path + '\Premium_data.csv'
df_premium.to_csv(filename, encoding='utf-8', index=False)        

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_premium.head(10)

2024-08-14 17:55:06.188683
svmem(total=16756752384, available=4355014656, percent=74.0, used=12401737728, free=4355014656)


In [9]:
%who_ls DataFrame 

['df_premium']

In [10]:
del df_premium


In [11]:
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#

2024-08-14 17:55:07.213715
svmem(total=16756752384, available=4424298496, percent=73.6, used=12332453888, free=4424298496)
